In [1]:
line="{{product_name:浙江在线杭州}}{{time:4月25日}}讯（记者{{person_name: 施宇翔}} 通讯员 {{person_name:方英}}）毒贩很“时髦”，用{{product_name:微信}}交易毒品。没料想警方也很“潮”，将计就计，一举将其擒获。记者从{{org_name:杭州江干区公安分局}}了解到，经过一个多月的侦查工作，{{org_name:江干区禁毒专案组}}抓获吸贩毒人员5名，缴获“冰毒”400余克，毒资30000余元，扣押汽车一辆。{{location:黑龙江}}籍男子{{person_name:钱某}}长期落脚于宾馆、单身公寓，经常变换住址。他有一辆车，经常半夜驾车来往于{{location:杭州主城区}}的各大宾馆和单身公寓，并且常要活动到{{time:凌晨6、7点钟}}，{{time:白天}}则在家里呼呼大睡。{{person_name:钱某}}不寻常的特征，引起了警方注意。禁毒大队通过侦查，发现{{person_name:钱某}}实际上是在向落脚于宾馆和单身公寓的吸毒人员贩送“冰毒”。"
entity=["time","location","person_name","org_name","company_name","product_name"]
import re,jieba


In [15]:
newline=[]
loc_entity=[]
person_entity=[]
org_entity=[]
doubelsurname=['欧阳', '太史', '端木', '上官', '司马', '东方', '独孤', '南宫', '万俟', '闻人', '夏侯', '诸葛', '尉迟', '公羊', '赫连', '澹台', '皇甫', '宗政', '濮阳', '公冶', '太叔', '申屠', '公孙', '慕容', '仲孙', '钟离', '长孙', '宇文', '司徒', '鲜于', '司空', '闾丘', '子车', '亓官', '司寇', '巫马', '公西', '颛孙', '壤驷', '公良', '漆雕', '乐正', '宰父', '谷梁', '拓跋', '夹谷', '轩辕', '令狐', '段干', '百里', '呼延', '东郭', '南门', '羊舌', '微生', '公户', '公玉', '公仪', '梁丘', '公仲', '公上', '公门', '公山', '公坚', '左丘', '公伯', '西门', '公祖', '第五', '公乘', '贯丘', '公皙', '南荣', '东里', '东宫', '仲长', '子书', '子桑', '即墨', '达奚', '褚师', '吴铭']
files='./NLP/NER/data/xinhua_dict.txt'
f=open(files,'r')
words=f.readlines()
f.close()

def append_entity(entity,newline):
    entity=list(jieba.cut(entity))
    for ent in entity:
        newline.append(ent)
    return newline,entity

def perEntity(entity):
    name=[]
    length=len(entity)
    if length!=1:
        if entity[0] not in doubelsurname and len(entity[0])==2:
            name.append("C")
            
context_u=False #上下文的上文
context_d=False #上下文的下文
compile=re.compile("{{(.*?)}}")
filter_compile=re.compile("\w+:")
split_line=compile.split(line)
for ind,lin in enumerate(split_line):
    if len(lin)==0:
        continue
    if filter_compile.match(lin):
        entity_name,entity=lin.split(":")
        newline,entity_cut=append_entity(entity,newline)
        entity_len=len(entity_cut)  
        if entity_name=='person_name':
            # 使用jieba分词，如果将人名分成一个单词，那么先判断前两个字是否为复姓，然后后面的按照正常的给予标签
            if entity_len==1:
                if entity[:2] in doubelsurname:
                    person_entity.append("C")
                    if len(entity[2:])==1:
                        person_entity.append("F")
                    else:
                        for _ in range(len(entity[2:-1])):
                            person_entity.append("D")
                        person_entity.append("E")
                elif entity[:2] in words and entity_len!=2:
                    person_entity.append("I")
                    if len(entity[2:])==1:
                        person_entity.append("F")
                    else:
                        for _ in range(len(entity[2:-1])):
                            person_entity.append("D")
                        person_entity.append("E")
                elif entity[:2] in words and entity_len==2:
                    person_entity.append("J")
                else:
                    person_entity.append("C")
                    if len(entity[1:])>=2:
                        for _ in range(len(entity[1:-1])):
                            person_entity.append("D")
                        person_entity.append("E")
                    else:
                        person_entity.append("F")
            else:
                if len(entity_cut[0])==2:
                    person_entity.append("I")
                context_d=True
        else:
            newline,_=append_entity(lin,newline)
            newline_len=len(newline)
            # 判断是否是连接词
            if context_d and filter_compile.match(split_line[ind+1]) and newline_len==1:
                person_entity.append("X")
                continue
            # 其他非连接词\n",
            if context_d and newline_len>1:
                person_entity.append("B")
                for _ in range(len(newline[1:-1])):
                    person_entity.append("Z")
                if filter_compile.match(split_line[ind+1]):
                    person_entity.append("A")
                else:
                    person_entity.append("Z")
                
        




Building prefix dict from the default dictionary ...


Dumping model to file cache /var/folders/83/ng05nj_s1_s9842rkp7xn5jc0000gp/T/jieba.cache


Loading model cost 1.737 seconds.


Prefix dict has been built succesfully.


['黑龙江']
['杭州', '主城区']


In [4]:
newline=[]
loc_entity=[]
org_entity=[]
doubelsurname=['欧阳', '太史', '端木', '上官', '司马', '东方', '独孤', '南宫', '万俟', '闻人', '夏侯', '诸葛', '尉迟', '公羊', '赫连', '澹台', '皇甫', '宗政', '濮阳', '公冶', '太叔', '申屠', '公孙', '慕容', '仲孙', '钟离', '长孙', '宇文', '司徒', '鲜于', '司空', '闾丘', '子车', '亓官', '司寇', '巫马', '公西', '颛孙', '壤驷', '公良', '漆雕', '乐正', '宰父', '谷梁', '拓跋', '夹谷', '轩辕', '令狐', '段干', '百里', '呼延', '东郭', '南门', '羊舌', '微生', '公户', '公玉', '公仪', '梁丘', '公仲', '公上', '公门', '公山', '公坚', '左丘', '公伯', '西门', '公祖', '第五', '公乘', '贯丘', '公皙', '南荣', '东里', '东宫', '仲长', '子书', '子桑', '即墨', '达奚', '褚师', '吴铭']
files='./NLP/NER/data/xinhua_dict.txt'
f=open(files,'r')
words=f.readlines()
f.close()

def append_entity(entity,newline):
    entity=list(jieba.cut(entity))
    for ent in entity:
        newline.append(ent)
    return newline,entity

def perEntity(entity):
    entity_tag=[]
    entity_par=[]
    entity_len=len(entity)
    # 对命名实体不使用jieba分词
    if entity_len==2:
        if entity in words:
            entity_tag.append("J-PER")
            entity_par.append(entity)
        else:
            entity_tag.extend(['C-PER','F-PER'])
            entity_par.extend(entity)
    else:
        if entity[:2] in doubelsurname:
            entity_tag.append('C-PER')
            entity_par.append(entity[:2])
            entity=entity[2:]
        elif entity[:2] in words:
            entity_tag.append("I-PER")
            entity_par.append(entity[:2])
            entity=entity[2:]
        else:
            entity_tag.append("C-PER")
            entity_par.append(entity[0])
            entity=entity[1:]
        if len(entity)==1:
            entity_tag.append("E-PER")
            entity_par.append(entity)
        elif len(entity)==2:
            if entity in words:
                entity_tag.append("K-PER")
                entity_par.append(entity)
            else:   
                entity_par.extend(entity)
                entity_tag.extend(["D-PER","E-PER"])
        else:
            raise(entity,'is longer than 2')
    return entity_tag,entity_par

context_u=False #上下文的上文
context_d=False #上下文的下文
compile=re.compile("{{(.*?)}}")
filter_compile=re.compile("\w+:")
split_line=compile.split(line)
words_seq=[]
person_entity=[]
for ind,lin in enumerate(split_line):
    if len(lin)==0:
        continue
    if filter_compile.match(lin):
        entity_name,entity=lin.split(":")
        if entity_name=='person_name':
            entity_tag,entity_par=perEntity(entity)
            context_d=True
        if entity_name=='location':
            
            pass
        else:
            newline,_=append_entity(lin,newline)
            newline_len=len(newline)
            # 判断是否是连接词
            if context_d and filter_compile.match(split_line[ind+1]) and newline_len==1:
                person_entity.append("X")
                continue
            # 其他非连接词
            if context_d and newline_len>1:
                person_entity.append("B")
                for _ in range(len(newline[1:-1])):
                    person_entity.append("Z")
                if filter_compile.match(split_line[ind+1]):
                    person_entity.append("A")
                else:
                    person_entity.append("Z")
                
        
    



['盛世良']

In [18]:
print(list(jieba.cut("海淀区")))

['海淀区']
